In [5]:
import numpy as np
from decisiontree import DecisionTreeRegressor

### Gradient Boosting Regression

> GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions. In each stage a regression tree is fit on the negative gradient of the given loss function.

> Initialize the model with a constant value: 

> $ 1. \; \;  F_0(x) =  arg \; min_\gamma \sum_{i=1}^n Loss(y_i, \gamma) = \frac{1}{n} \sum_{i=1}^n y_i  $

> For m = 1 to M,

> $ 2. \; \; \text{Compute Psuedo Residuals, } r_{im} = - [ \frac{ \partial Loss( y_i , F(x_i) ) }{ \partial F(x_i) } ]_{F(x) = F_{m-1}(x)} \;\;\; \text{for i = 1,2...n} $

>    $ \; \; \; \;\text{For MSE Loss, psuedo residuals } r_{i} = - \frac{\partial(y_i-\hat y_i)^2}{\partial \hat y_i} = - (-2) ( y_i - \hat y_i ) = 2( y_i - \hat y_i) $ 

> $ 3. \; \; \text{Fit base trainer, } h_m(x) \text{ to psuedo residuals} $

> $ 4. \; \; \text{Update } F_m(x) = F_{m-1}(x) + \gamma_m h_m(x) $

> $ \; \; \; \; \; \gamma_m \text{ step-size is choosen using line search, } \gamma_m = arg\;min_\gamma \sum_{i=1}^n L(y_i,F_{m-1}(x_i)) - \gamma \frac{\partial L(y_i,F_{m-1}(x_i))}{\partial F_{m-1}(x_i)}$

> $$ \; \; \text{Final model will be, } F(x) = \frac{1}{n} \sum_{i=1}^n y_i + \sum_{m=1}^M \gamma_m h_m(x) $$


* In code below, step size is constant.
* [Nice Article](https://scikit-learn.org/stable/modules/ensemble.html#gradient-tree-boosting)
* [Nice Article](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/?source=post_page-----1e317ae4587d----------------------)

In [22]:
class GradientBoostingRegressor():
    def __init__(self, loss='ls', learning_rate = 0.1, n_estimators=100, criterion='mse', 
                 max_depth=None, min_samples_split=2, max_features=None, verbose=False):
        self.__lr = learning_rate
        self.__n_estimators = n_estimators
        self.__criterion = criterion
        self.__max_depth = max_depth
        self.__min_samples_split = min_samples_split
        self.__max_features = None
        if isinstance(max_features,str):
            self.__max_features = { 
            'auto': lambda x: int(np.sqrt(x)), 'sqrt': lambda x: int(np.sqrt(x)), 
            'log2': lambda x: int(np.log2(x)), 'max_features': lambda x: x  }[max_features]
        elif isinstance(max_features, int):
            self.__max_features = lambda x: max_features
        elif isinstance(max_features, float):
            self.__max_features = lambda x: int(max_features*x)
        else:
            self.__max_features = lambda x: x
            
        self.__n_features = None
        self.__trees = []
        self.__verbose = verbose
        self.__f0 = None
    
    def __mse(self,y_pred,y_true):
        return np.sqrt( np.mean( (y_true-y_pred)**2 ) )
    
    def __negative_least_squares_gradient(self,y_pred,y_true):
        grad =  -2 * (y_true - y_pred)
        return -1 * grad
    def __get_feature_index(self): 
        return np.random.choice( np.arange(0,self.__n_features,1), 
                                size=self.__max_features(self.__n_features), replace=False)
    
    def fit(self, X, y):
        self.__n_features = X.shape[1]
        y_ = self.__f0 = y.mean()
        for i in range(0,self.__n_estimators):
            dt = DecisionTreeRegressor(criterion=self.__criterion, 
                                       max_depth=self.__max_depth, 
                                       min_samples_split=self.__min_samples_split)
            feature_index = self.__get_feature_index()
            h = self.__negative_least_squares_gradient(y_,y)
            dt.fit(X[:,feature_index], h)
            self.__trees.append( (dt.tree_,feature_index) )
            y_ = self.predict(X)
            if self.__verbose and i%5==0:
                print( f"MSE after trees {i+1} : {self.__mse(y_,y)}" )
            
    def predict(self, X):
        predictions = np.ones( len(X) ) * self.__f0
        for i in range(1,len(self.__trees)+1):
            root, features = self.__trees[i-1]
            predictions += self.__lr * np.array([ self.__predict_row(row,root) for row in X[:,features] ])
        return predictions
            
    def __predict_row(self,row,node):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict): return self.__predict_row(row,node['left'])
            else: return node['left']
        else:
            if isinstance(node['right'], dict): return self.__predict_row(row,node['right'])
            else: return node['right']
    
    def score(self,X,y):
        y_pred = self.predict(X)
        return 1 - np.sum(np.square(y-y_pred))/np.sum(np.square(y-y.mean()))

In [23]:
import pandas as pd
data = pd.read_csv('data.csv',).values
X = data[:,:-1]
y = data[:,-1]
X.shape,y.shape

((506, 13), (506,))

In [24]:
from sklearn.model_selection import train_test_split
X_t,X_v,Y_t,Y_v = train_test_split(X,y,test_size=0.3)

In [30]:
gbr = GradientBoostingRegressor(learning_rate=0.1,
                                n_estimators=20,
                               verbose=False)
gbr.fit(X_t,Y_t)
gbr.score(X_t,Y_t),gbr.score(X_v,Y_v)

(0.9998620510817293, 0.8385563972264015)